### 볼린저 밴드 투자 전략
1. 이동 평균선 생성 : 데이터의 20개의 평균을 구한 값
2. 상단 밴드 생성 : 이동 평균선 + ( 2 * 데이터 20개의 표준편차 )
3. 하단 밴드 생성 : 이동 평균선 - ( 2* 데이터 20개의 표준편차 )
4. 하단 밴드보다 주식의 가격이 낮은 경우 구매 
5. 상단 밴드보다 주식의 가격이 높은 경우 판매

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# 데이터 로드 
df = pd.read_csv('../../csv/AAPL.csv')

In [ ]:
# Date컬럼을 시계열 데이터로 변경 
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# 결측치나 아니면 무한대 값을 제거 
flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
df = df.loc[~flag]

In [ ]:
# Date 컬럼을 인덱스로 변경 
df.set_index('Date', inplace=True)

In [ ]:
# 이동 평균선 : 데이터 20개의 평균 값
# 이동 평균선 컬럼을 하나 생성 
df['center'] = np.nan

In [ ]:
mean_data = df.iloc[0:20]['Close'].mean()
df.iloc[19, 6] = mean_data

In [ ]:
df.iloc[19]

In [ ]:
for i in range(20, len(df), 1):
    mean_data = df.iloc[i-20 : i]['Close'].mean()
    df.iloc[i-1, 6] = mean_data

df.iloc[18 : 25]

In [ ]:
# df에서 Adj Close의 값만 빼고 모두 제거 
price_df = df[['Adj Close']]
price_df.head()

In [ ]:
# 이동 평균선 생성 : 
# rolling(n) : n개 만큼 데이터를 그룹화하여 연산식을 사용
price_df['center'] = price_df['Adj Close'].rolling(20).mean()

In [ ]:
price_df.iloc[18:24]

In [ ]:
price_df['index'] = range(1, len(price_df)+1)

In [ ]:
price_df.iloc[18:24]

In [ ]:
price_df['Adj Close'].rolling(20).std() * 2

In [ ]:
# 상단 밴드 (이동 평균선 + (2 * 20개의 데이터의 표준편차)) , 하단 밴드를 생성 
price_df['ub'] = price_df['center'] + \
    (2 * price_df['Adj Close'].rolling(20).std())

price_df['lb'] = price_df['center'] - \
    (2 * price_df['Adj Close'].rolling(20).std())

In [ ]:
price_df.iloc[18:24]

In [ ]:
# price_df에서 하단의 1000개의 데이터
# center, ub, lb 데이터를 plot 그래프 표시 
price_df.tail(1000)

plt.figure(figsize=(20, 8))
plt.plot(price_df[['ub', 'lb', 'Adj Close']].tail(1000))
plt.legend(['ub', 'lb', 'Adj Close'])
plt.show()

In [ ]:
# 투자 기간을 선택 
start = "2000-01-01"

start = datetime.strptime(start, '%Y-%m-%d')

test_df = price_df.loc[start:]

In [ ]:
# 보유 내역 컬럼을 생성 
test_df['trade'] = ""

### 보유 내역 추가 
- 조건식 
    - 상단 밴드보다 수정 주가가 높은 경우 
        - 현재 보유 상태라면
            - 매도 (trade = "")
        - 현재 보유중이 아니라면
            - 아무 행동도 하지 않는다. (trade ="")
    - 상단 밴드와 하단 밴드 사이에 수정 주가가 존재하는 경우
        - 현재 보유 상태라면
            - 아무 행동도 하지 않는다. (trade = "buy")
        - 현재 보유중이 아니라면 
            - 아무 행동도 하지 않는다. (trade = "")
    - 하단 밴드보다 수정 주가가 낮은 경우
        - 현재 보유 상태라면
            - 아무 행동도 하지 않는다 (trade = 'buy')
        - 현재 보유중이 아니라면
            - 매수 (trade = 'buy')

In [ ]:
test_df.head(1)

In [ ]:
for i in test_df.index:
    # i는 test_df의 index 값들이 대입 

    # 상단 밴드보다 수정 주가가 높은 경우
    if test_df.loc[i, 'Adj Close'] > test_df.loc[i, 'ub']:
        # 현재 보유 중이라면? -> 전날의 trade가 buy라면
        if test_df.shift().loc[i, 'trade'] == 'buy':
            # 매도 
            test_df.loc[i, 'trade'] = ""
        else:
            # 아무 행동도 하지 않는다. 
            test_df.loc[i, 'trade'] = ""
    # 하단 밴드보다 수정 주가가 낮은 경우 
    elif test_df.loc[i, 'Adj Close'] < test_df.loc[i, 'lb']:
        # 현재 보유 상태라면?
        if test_df.shift().loc[i, 'trade'] == "buy":
            # 아무 행동도 하지 않는다. 
            test_df.loc[i, 'trade'] = "buy"
        else:
            # 매수
            test_df.loc[i, 'trade'] = "buy"
    # 밴드 사이에 수정 주가가 존재하는 경우
    else:
        # 현재 보유 상태라면?
        if test_df.shift().loc[i, 'trade'] == 'buy':
            # 아무 행동도 하지 않는다. 
            test_df.loc[i, 'trade'] = "buy"
        else:
            # 아무 행동도 하지 않는다. 
            test_df.loc[i, 'trade'] = ""
        

In [ ]:
test_df['trade'].value_counts()

### 수익율 계산
- 구매한 날의 수정 주가 
    - 전날의 trade가 "" 오늘의 trade가 "buy"인 날의 수정 주가
- 판매한 날의 수정 주가 
    - 전날의 trade가 "buy" 오늘의 trade가 ""인 날의 수정 주가 
- 수익율 
    - 판매한 날의 수정 주가 / 구매한 날의 수정 주가 

In [ ]:
# 수익율 파생변수를 생성 
test_df['rtn'] = 1

for i in test_df.index:
    # 구매가를 대입 
    if (test_df.shift().loc[i, 'trade'] == "") & \
        (test_df.loc[i, 'trade'] == "buy"):
        buy = test_df.loc[i, 'Adj Close']
        print(f'매수 일 : {i}, 매수가 : {buy}')
    # 판매가를 대입 
    elif (test_df.shift().loc[i, 'trade'] == "buy") & \
        (test_df.loc[i, 'trade'] == ""):
        sell = test_df.loc[i, 'Adj Close']
        # 수익율 계산
        rtn = sell / buy
        test_df.loc[i, 'rtn'] = rtn
        print(f'매도 일 : {i}, 매도가 : {sell}, 수익율 : {rtn}')

In [ ]:
# 누적 수익율
acc_rtn = 1

for i in test_df.index:
    rtn = test_df.loc[i, 'rtn']
    acc_rtn *= rtn

acc_rtn

In [ ]:
test_df['acc_rtn'] = test_df['rtn'].cumprod()

In [ ]:
test_df.tail()

### 볼린저 밴드를 함수화
1. 1번 함수(밴드 생성) create_band()
    - 매개변수 4개 (데이터프레임, 기준이 되는 컬럼명, 시작시간, 종료시간)
    - 인덱스가 Date인지를 확인하여 인덱스가 Date가 아니면 Date 컬럼을 인덱스로 변환
    - 인덱스를 시계열 데이터로 변경
    - 기준이 되는 컬럼을 제외하고 모두 삭제
    - 결측치, 이상치(무한대) 값들을 제거 
    - 이동 평균선, 상단 밴드, 하단 밴드 파생변수를 생성
    - 시간시간과 종료시간으로 데이터를 필터링 
    - 위의 과정에서 나온 데이터프레임을 리턴 
2. 2번 함수(트레이드 생성) create_trade()
    - 매개변수가 1개 (데이터프레임) -> 1번 과정에서 나온 결과값 대입
    - trade 라는 파생변수를 생성하여 데이터는 ""로 대입
    - 볼린저밴드를 이용하여 거래 내역을 추가 
    - 결과값을 리턴
3. 3번 함수( 수익율 계산 ) create_rtn()
    - 매개변수는 1개 (데이터프레임) -> 2번 과정에서 나온 결과값 대입
    - 수익율 파생변수 생성하여 데이터는 1로 대입
    - 구매한 날의 데이터와 판매한 날의 데이터를 가지고 수익율을 생성하여 대입
    - 누적 수익율을 계산하여 새로운 파생변수에 대입 
    - 최종 누적 수익율을 print을 이용하여 출력
    - 결과(데이터프레임)를 리턴

In [ ]:
# 첫번째 함수 생성 
def create_band(
        _df, 
        _col = 'Adj Close', 
        _start="2010-01-01", 
        _end = '2023-12-31', 
        _roll = 20
    ):
    # 컬럼에 Date가 존재한다면
    if 'Date' in _df.columns:
        _df.set_index('Date', inplace=True)
    # index를 시계열 변경 
    _df.index = pd.to_datetime(_df.index)

    # 특정 컬럼의 제외하고 모두 제거 
    price_df = _df[[_col]]
    # 결측치, 무한대 값이 존재하는 인덱스 조건식
    flag = price_df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    # 결측치, 무한대를 제거 
    price_df = price_df.loc[~flag]

    # 이동 평균선 생성
    price_df['center'] = price_df[_col].rolling(_roll).mean()
    # 상단 밴드 생성
    price_df['ub'] = price_df['center'] + (2 * price_df[_col].rolling(_roll).std())
    # 하단 밴드 생성
    price_df['lb'] = price_df['center'] - (2 * price_df[_col].rolling(_roll).std())

    # 시작시간과 종료 시간을 기준으로 필터링 
    start = datetime.strptime(_start, '%Y-%m-%d')
    end = datetime.strptime(_end, '%Y-%m-%d')
    price_df = price_df.loc[start:end]
    return price_df

In [ ]:
df = pd.read_csv("../../csv/AAPL.csv")

In [ ]:
df2 = create_band(df, 'Close', '2000-01-01', '2023-12-04')
df2.tail()

In [ ]:
df2.columns[0]

In [ ]:
# 두번째 함수 
def create_trade(_df):
    # 첫번째 함수에서 지정한 컬럼의 이름이 무엇인가? -> _df의 컬럼중 첫번째 데이터
    col = _df.columns[0]

    # 거래 내역 컬럼을 추가 
    _df['trade'] = ""

    # 거래 내역을 추가 
    for i in _df.index:
        # 상단밴드보다 col의 값이 높은 경우
        if _df.loc[i, col] > _df.loc[i, 'ub']:
            _df.loc[i, 'trade'] = ""
        # 하단밴드보다 col의 값이 낮은 경우
        elif _df.loc[i, col] < _df.loc[i, 'lb']:
            _df.loc[i, 'trade'] = 'buy'
        # col의 값이 밴드 사이에 존재한다면
        else:
            # 보유 상태라면 
            if _df.shift().loc[i, 'trade'] == 'buy':
                _df.loc[i, 'trade'] = 'buy'
            else:
                _df.loc[i, 'trade'] = ''
        
    return _df
    

In [ ]:
df3 = create_trade(df2)
df3.tail()

In [70]:
df3['trade'].value_counts()

trade
       3080
buy    1819
Name: count, dtype: int64

In [71]:
# 세번째 함수 생성 
def create_rtn(_df):
    # 기준이 되는 컬럼의 이름 
    col = _df.columns[0]
    # 수익율 파생변수 생성 데이터는 1로 대입
    _df['rtn'] = 1

    # 수익율 대입 
    for i in _df.index:
        # 구입
        if (_df.shift().loc[i, 'trade'] == "") & \
            (_df.loc[i, 'trade'] == "buy"):
            buy = _df.loc[i, col]
            print(f'매수일 : {i}, 매수가 : {buy}')
        # 판매
        elif (_df.shift().loc[i, 'trade'] == "buy") & \
            (_df.loc[i, 'trade'] == ""):
            sell = _df.loc[i, col]
            # 수익율 발생
            rtn = sell / buy
            # 수익율 대입 
            _df.loc[i, 'rtn'] = rtn
            # 출력 
            print(f'매도일 : {i}, 매도가 : {sell}, 수익율 : {rtn}')
    _df['acc_rtn'] = _df['rtn'].cumprod()
    # 최종 누적수익율을 출력
    print(_df['acc_rtn'][-1])
    return _df

In [73]:
create_rtn(df3)

매수일 : 2000-01-12 00:00:00, 매수가 : 3.113839
매도일 : 2000-01-20 00:00:00, 매도가 : 4.053571, 수익율 : 1.3017920965085221
매수일 : 2000-04-12 00:00:00, 매수가 : 3.901786
매도일 : 2000-06-20 00:00:00, 매도가 : 3.616071, 수익율 : 0.9267732776733526
매수일 : 2000-07-24 00:00:00, 매수가 : 3.477679
매도일 : 2000-08-23 00:00:00, 매도가 : 3.879464, 수익율 : 1.1155325146455437
매수일 : 2000-09-22 00:00:00, 매수가 : 3.727679
매도일 : 2001-01-03 00:00:00, 매도가 : 1.169643, 수익율 : 0.31377245733873543


C:\Users\moons\AppData\Local\Temp\ipykernel_27060\1226163313.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.3017920965085221' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  _df.loc[i, 'rtn'] = rtn


매수일 : 2001-05-30 00:00:00, 매수가 : 1.412857
매도일 : 2001-06-25 00:00:00, 매도가 : 1.713571, 수익율 : 1.212841073088076
매수일 : 2001-07-18 00:00:00, 매수가 : 1.485
매도일 : 2001-10-11 00:00:00, 매도가 : 1.267143, 수익율 : 0.8532949494949493
매수일 : 2002-02-21 00:00:00, 매수가 : 1.535714
매도일 : 2002-04-16 00:00:00, 매도가 : 1.838571, 수익율 : 1.1972092459924177
매수일 : 2002-04-26 00:00:00, 매수가 : 1.643571
매도일 : 2002-10-25 00:00:00, 매도가 : 1.101429, 수익율 : 0.6701438514064803
매수일 : 2002-12-05 00:00:00, 매수가 : 1.045
매도일 : 2003-02-18 00:00:00, 매도가 : 1.090714, 수익율 : 1.0437454545454545
매수일 : 2003-04-11 00:00:00, 매수가 : 0.942857
매도일 : 2003-05-05 00:00:00, 매도가 : 1.149286, 수익율 : 1.2189398816575578
매수일 : 2003-06-09 00:00:00, 매수가 : 1.199286
매도일 : 2003-06-18 00:00:00, 매도가 : 1.365714, 수익율 : 1.138772569678959
매수일 : 2003-09-24 00:00:00, 매수가 : 1.522857
매도일 : 2004-01-05 00:00:00, 매도가 : 1.583571, 수익율 : 1.0398684840401955
매수일 : 2004-04-30 00:00:00, 매수가 : 1.841429
매도일 : 2004-05-25 00:00:00, 매도가 : 2.029286, 수익율 : 1.102016966171381
매수일 : 2005-04-14 00

C:\Users\moons\AppData\Local\Temp\ipykernel_27060\1226163313.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(_df['acc_rtn'][-1])


,Close,center,ub,lb,trade,rtn,acc_rtn
Date,,,,,,,
2000-01-03,3.997768,3.675893,4.126560,3.225225,,1.0,1.00000
2000-01-04,3.660714,3.651786,4.045227,3.258344,,1.0,1.00000
2000-01-05,3.714286,3.627121,3.923805,3.330436,,1.0,1.00000
2000-01-06,3.392857,3.600223,3.877917,3.322530,,1.0,1.00000
2000-01-07,3.553571,3.589955,3.857958,3.321953,,1.0,1.00000
...,...,...,...,...,...,...,...
2019-06-18,198.449997,185.432500,201.032574,169.832427,buy,1.0,1.13928
2019-06-19,197.869995,185.996000,202.558154,169.433846,buy,1.0,1.13928
2019-06-20,199.460007,186.830000,204.361771,169.298229,buy,1.0,1.13928
